In [2]:
# https://platform.olimpiada-ai.ro/problems/59

import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json

In [5]:
with open('/kaggle/input/disney-festival/train.json', 'r') as f:
    train = json.load(f)

with open('/kaggle/input/disney-festival/test.json', 'r') as f:
    test = json.load(f)

len(train), len(test)

(200, 100)

In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("zero-shot-image-classification", model="openai/clip-vit-base-patch32")

2025-12-14 13:52:09.401280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765720329.657901      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765720329.733168      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


In [20]:
candidate_labels = ['Pokemon', 'Tarzan', 'Snow White', 'Winnie the Pooh']

out = pipe(os.path.join('/kaggle/input/disney-festival', train[0]['image_path']), candidate_labels=candidate_labels)
choice = out[0]['label']

print(choice, train[0]['cartoon_class'], sep=', ')

Pokemon, Pokemon


In [35]:
correct, total = 0, 0

for i, sample in (pbar := tqdm(enumerate(train), total=len(train))):
    out = pipe(os.path.join('/kaggle/input/disney-festival', sample['image_path']), candidate_labels=candidate_labels)
    choice = out[0]['label']

    if sample['cartoon_class'] == choice:
        correct += 1
    total += 1

    pbar.set_postfix({'accuracy': f'{correct/total:.5f}'})

print(f'Accuracy: {correct/total:.5f}')

  0%|          | 0/200 [00:00<?, ?it/s]

Accuracy: 1.00000


In [36]:
paths, preds = [], []

for i, sample in tqdm(enumerate(test), total=len(test)):
    paths.append(sample['image_path'])
    
    out = pipe(os.path.join('/kaggle/input/disney-festival', sample['image_path']), candidate_labels=candidate_labels)
    choice = out[0]['label']

    preds.append(choice)

subm = pd.DataFrame({
    'image_path': paths,
    'cartoon_class': preds
})

subm.to_csv("submission.csv", index=False)

subm.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,image_path,cartoon_class
0,images/test_0001.png,Pokemon
1,images/test_0002.png,Pokemon
2,images/test_0003.png,Pokemon
3,images/test_0004.png,Tarzan
4,images/test_0005.png,Tarzan
